# Learning Econometrics in a unique way:
So far, I have just been reading the assumptions and statements given in the textbook as facts, and never actually stopped to question myself 'Why'?
Hence, here are a few ideas that I'd be covering in this jupiter file:
- Stating the assumptions (Simple and Multiple linear regressions)and actually testing them using fabricated sample data/datasets found on kaggle
- Creating a table to show clear calculation of the residuals, error terms, etc
- Creating neat regression tables with p-values shown and interpreting these values for fun regressions (like impact of taylor swift's songs on peoples' moods, movie dataset, etc.)

My ultimate goal is learn this subject properly, and also share this knowledge to people around the world.
In this goal, I wish to cover:
- Handling and setting up my data properly for regressions
- Writing clean lines of code with proper comments
- Creating great visualizations using matplotlib and seaborn
- (Maybe in the future)Creating animated plots using dash for people to understand econometrics even better